<a href="https://colab.research.google.com/github/gordenbuh/ClimaMetrics/blob/main/pruebas_hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- (Código anterior: Instalación de librerías, montaje de Drive, definición de rutas,
# ---  carga/simulación de datos, preprocesamiento, entrenamiento del modelo de ML,
# ---  carga del modelo de Hugging Face, función generar_informe_alerta_hf) ---

# --- (Reutilizamos el código anterior hasta la parte de integración con el LLM) ---

# --- Preparación del Entorno en Google Colab ---

# 1. Instalación de librerías (¡importante en Colab!)
!pip install pandas numpy scikit-learn matplotlib seaborn transformers torch

# 2. Montar Google Drive (opcional, pero MUY recomendado)
from google.colab import drive
drive.mount('/content/drive')

# 3. Definir rutas (ajustar a tu estructura de carpetas en Drive)
#    Si no usas Google Drive, comenta estas líneas y ajusta las rutas a archivos locales
project_path = '/content/drive/MyDrive/MiProyectoHackathon/'  # Cambiar a tu ruta
data_path = os.path.join(project_path, 'datos/')
model_path = os.path.join(project_path, 'modelos/')

# Crea los directorios si no existen
os.makedirs(data_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import os
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:


# --- 1. RECOPILACIÓN Y PREPARACIÓN DE DATOS ---

# Simulación de datos (reemplazar con tus datos reales)
# Si tienes datos reales, cárgalos desde Drive (o desde donde los tengas)
# Ejemplo (si tienes un CSV en Drive):
# df = pd.read_csv(os.path.join(data_path, 'tus_datos.csv'))

np.random.seed(42)
num_samples = 1000
data = {
    'precipitacion_24h': np.random.rand(num_samples) * 200,
    'precipitacion_72h': np.random.rand(num_samples) * 300,
    'nivel_rio': np.random.rand(num_samples) * 5 + np.random.rand(num_samples) * 2 * (np.random.rand(num_samples) > 0.7),
    'humedad_suelo': np.random.rand(num_samples) * 0.6 + 0.4 * (np.random.rand(num_samples) > 0.8),
    'temperatura': np.random.rand(num_samples) * 20 + 10,
    'inundacion': np.zeros(num_samples)
}
for i in range(num_samples):
    if (data['precipitacion_24h'][i] > 100 and data['nivel_rio'][i] > 4) or \
       (data['precipitacion_72h'][i] > 200 and data['humedad_suelo'][i] > 0.8) or \
       (data['nivel_rio'][i] > 6):
        data['inundacion'][i] = 1

df = pd.DataFrame(data)



In [ ]:


print(df.head())
print(df.describe())
print(df['inundacion'].value_counts())  # Verificar el balance de clases


In [ ]:

# Visualización de datos para ver la proporcion de 1/0
sns.pairplot(df, hue='inundacion', diag_kind='kde')
plt.show()


In [ ]:
# Definición de las variables de desarrollo

X = df.drop('inundacion', axis=1)
y = df['inundacion']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


# --- 2. DESARROLLO DEL MODELO DE ML ---

# Modelo de ML (ejemplo con Random Forest, pero puedes usar otros)
model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)  # Ajusta hiperparámetros
# n_estimators: número de árboles.  Más árboles = mejor, pero más lento.
# max_depth: profundidad máxima de los árboles.  Evita overfitting.

model.fit(X_train_scaled, y_train)

# --- Guardar el modelo de ML (en Google Drive, si lo has montado) ---
import joblib  # Para guardar modelos de scikit-learn
model_filename = os.path.join(model_path, 'modelo_ml.joblib')  # Ruta completa
joblib.dump(model, model_filename)
print(f"Modelo de ML guardado en: {model_filename}")

# --- (Si quieres cargar el modelo más tarde) ---
# model = joblib.load(model_filename)



In [ ]:

# --- 3. INTEGRACIÓN CON EL LLM (Hugging Face) ---

# --- Configuración del LLM ---

# Elige tu modelo
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # O el que prefieras

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_llm = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",  # Usa GPU si está disponible
        torch_dtype=torch.float16,  # Usa float16 para ahorrar memoria (si es compatible)
        # low_cpu_mem_usage=True  # Reduce el uso de CPU RAM (útil en Colab)
    )

    pipe = pipeline(
        "text-generation",
        model=model_llm,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        num_return_sequences=1,
        # batch_size=8,  # Ajusta el tamaño del batch si tienes problemas de memoria (¡importante en Colab!)
    )


except Exception as e:
    print(f"Error al cargar el modelo '{model_name}' de Hugging Face: {e}")
    raise


In [ ]:
def generar_informe_alerta_hf(variables_criticas, prediccion, ubicacion="una zona de prueba"):
    prompt = f"""
Contexto: Estamos en {ubicacion}. El modelo de predicción de inundaciones ha identificado las siguientes
variables como las más influyentes:
"""
    for var, valor in variables_criticas.items():
        prompt += f"- {var}: {valor}\n"
    prompt += f"El modelo predice una probabilidad de inundación de {prediccion:.2f}.\n\n"
    prompt += """
Instrucción: Genera un informe de alerta de inundación conciso y fácil de entender para el público en general. Incluye:
- Nivel de alerta (bajo, medio, alto).
- Áreas potencialmente afectadas (se específico en la medida de lo posible basándote en la ubicación y el nivel de riesgo, pero no inventes nombres si no los tienes).
- Recomendaciones de seguridad (evacuación, preparación, etc.).  Sé lo más específico posible, adaptando las recomendaciones al nivel de alerta.
- Hora y fecha de emisión de la alerta.
- Un llamado a la acción. (Por ejemplo: "Manténgase informado a través de los canales oficiales").
Formato: Usa viñetas para cada sección del informe.
"""
    try:
      #Para modelos que usan un formato de chat (ej: Mistral)
      if "mistralai" in model_name.lower() or "llama" in model_name.lower():
        formatted_prompt = f"<s>[INST] {prompt} [/INST]"
        result = pipe(formatted_prompt)
        informe = result[0]['generated_text'].replace(formatted_prompt, "").strip()
      # Si usas un modelo tipo T5 (como Flan-T5)
      elif "t5" in model_name.lower(): #Para modelos que usan una estructura Input/Output como T5
          input_prompt = f"Genera informe de alerta: {prompt}"
          result = pipe(input_prompt)
          informe = result[0]['generated_text'].strip()

      else: #En caso de usar otro modelo, se puede intentar una aproximación
          result = pipe(prompt)
          informe = result[0]['generated_text'].strip()
    except Exception as e:
        print(f"Error al generar el informe con el modelo de Hugging Face: {e}")
        informe = "No se pudo generar el informe debido a un error."

    return informe


In [ ]:


# --- INFORME PARA EL REGISTRO CON MAYOR PROBABILIDAD ---

# 1. Predecir probabilidades para todos los registros de prueba
probabilidades = model.predict_proba(X_test_scaled)[:, 1]  # Probabilidad de la clase 1 (inundación)

# 2. Encontrar el índice del registro con la probabilidad más alta
indice_max_prob = np.argmax(probabilidades)

# 3. Obtener los datos de ESE registro
registro_max_prob = X_test.iloc[indice_max_prob]
probabilidad_max = probabilidades[indice_max_prob]


# 4. Obtener las variables más importantes para el informe (usando el modelo de ML)
if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
    feature_names = X.columns
    indices = np.argsort(importances)[::-1]
    top_variables = [feature_names[i] for i in indices[:3]]  # Top N variables
    variables_criticas = {var: registro_max_prob[var] for var in top_variables}
elif hasattr(model, 'coef_'):
    importances = np.abs(model.coef_[0])
    feature_names = X.columns
    indices = np.argsort(importances)[::-1]
    top_variables = [feature_names[i] for i in indices[:3]]
    variables_criticas = {var: registro_max_prob[var] for var in top_variables}
else:  # Si el modelo no proporciona importancia de variables
    variables_criticas = {
        'precipitacion_24h': registro_max_prob['precipitacion_24h'],
        'nivel_rio': registro_max_prob['nivel_rio'],
        'humedad_suelo': registro_max_prob['humedad_suelo']
    }
    top_variables = list(variables_criticas.keys())


# 5. Generar el informe para el registro con mayor riesgo
informe_especifico = generar_informe_alerta_hf(
    variables_criticas, probabilidad_max, ubicacion="la zona de pruebas"
)

print("\n--- INFORME DE ALERTA (Registro con Mayor Probabilidad) ---")
print(informe_especifico)



In [ ]:

# --- (Opcional) Sistema de Alerta (simulación) ---
# (El código del sistema de alerta es el mismo, pero ahora podrías usar
#  informe_especifico en lugar de un informe genérico)
def enviar_alerta(informe, destinatarios):
    print(f"\n--- Enviando Alerta ---")
    print(f"Destinatarios: {', '.join(destinatarios)}")
    print(f"Mensaje:\n{informe}")
    print("Alerta enviada (simulación).")
destinatarios_prueba = ["+1234567890", "usuario1@example.com", "usuario2@example.com"]
enviar_alerta(informe_especifico, destinatarios_prueba) #Usamos el informe especifico.